In [3]:
import json
from pathlib import Path

加载和合并原始数据

In [31]:
data_dir = Path('../data')

In [32]:
ccf = json.load(open(data_dir / 'CCF.json', 'r', encoding='utf-8'))
len(ccf)

200000

In [33]:
sci = json.load(open(data_dir / 'CS-SCI_1-2.json', 'r', encoding='utf-8'))
len(sci)

100000

In [34]:
with open(data_dir / 'arxiv.json', 'r', encoding='utf-8') as f:
    arxiv = [json.loads(line.strip(',\n')) for line in f.readlines()]
len(arxiv)

200000

In [7]:
from itertools import chain
from tqdm import tqdm

In [36]:
data = list(chain(ccf, sci, arxiv))
len(data)

500000

In [37]:
del ccf, sci, arxiv

所有数据共50万条，每条数据均包含_id、title、abstract、author四个字段

数据清洗
1. 去除缺失值
2. 去除重复值

In [38]:
data1 = list(filter(lambda d: d['_id'] and d['title'] and d['abstract'] and d['author'], data))
len(data1)

491862

In [39]:
unique_id = set()
unique_title = set()
data2 = []
for d in tqdm(data1):
    if d['_id'] not in unique_id and d['title'] not in unique_title:
        unique_id.add(d['_id'])
        unique_title.add(d['title'])
        data2.append(d)

len(data2)

100%|██████████| 491862/491862 [00:00<00:00, 1893975.74it/s]


449383

In [43]:
for d in tqdm(data2):
    d['_id'] = d['_id'].replace('\n', '')
    d['title'] = d['title'].replace('\n', '')
    d['abstract'] = d['abstract'].replace('\n', ' ')
    d['author'] = d['author'].replace('\n', '')

100%|██████████| 449383/449383 [00:00<00:00, 1151321.36it/s]


In [44]:
with open(data_dir / 'data.jsonl', 'w', encoding='utf-8') as f:
    for d in tqdm(data2):
        f.write(json.dumps(d, ensure_ascii=False) + '\n')

100%|██████████| 449383/449383 [00:13<00:00, 32194.50it/s]


In [11]:
prompt = '''
# 任务：从给定的论文摘要文本中提取命名实体和实体之间的关系。
# 任务步骤：
1. 归纳总结出摘要文本中的研究问题(Problem)、实验结果(Result)和结论(Conclusion)。
2. 分别从总结的文本中提取出对应的一个或多个关键词，每个关键词表示一个实体，它的类型必须是Problem、Result和Conclusion中的一个，每个实体的id必须为一个数字
3. 严格按规定的输出格式输出
4. 必须检查输出结果
# 输入：{0}
# 输出格式：
{{
    'entities': [
        {{'id': '', 'type': '', 'name': ''}},
        {{'id': '', 'type': '', 'name': ''}},
        {{'id': '', 'type': '', 'name': ''}},
        {{'id': '', 'type': '', 'name': ''}}
    ],
    'relations': [
        {{'type': '', 'from_entity_id': '', 'to_entity_id': '', 'from_entity_type': '', 'to_entity_type': ''}},
    ]
}}
'''

In [12]:
with open('../data/data.json', 'r', encoding='utf8') as f:
    data = json.loads(f.read())
    data = [
        {
            "custom_id": d['_id'],
            "method": "POST",
            "url": "/v4/chat/completions",
            "body": {
                "model": "glm-4-flash",
                "messages": [
                    {"role": "system", "content": "你好，你是一个提取文本实体和关系的专家。"},
                    {"role": "user", "content": prompt.format(d['abstract'])}
                ],
            }
        }
        for d in data
    ]

with open('../data/data_prompt1.jsonl', 'w', encoding='utf8') as f:
    for d in data:
        f.write(json.dumps(d, ensure_ascii=False) + '\n')
